<a href="https://colab.research.google.com/github/phongpirik/CS2225.CH1501/blob/tungnqd/remover_Background.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [2]:
#!/bin/bash
!wget http://download.tensorflow.org/models/deeplabv3_mnv2_pascal_train_aug_2018_01_29.tar.gz
!wget http://download.tensorflow.org/models/deeplabv3_pascal_train_aug_2018_01_04.tar.gz

!mkdir -p models/mobile_net_model/model
!tar xvzf deeplabv3_mnv2_pascal_train_aug_2018_01_29.tar.gz -C models/mobile_net_model/model --strip=1

!mkdir -p models/xception_model/model
!tar xvzf deeplabv3_pascal_train_aug_2018_01_04.tar.gz -C models/xception_model/model --strip=1


!mkdir -p models/basnet
!mkdir -p models/u2net
!mkdir -p models/u2netp

!cd setup/
!/usr/bin/python3 download.py

#!mv basnet.pth ../models/basnet/
#!mv u2net.pth ../models/u2net/
#!mv u2netp.pth ../models/u2netp/

#!cd ../
#!rm deeplabv3_mnv2_pascal_train_aug_2018_01_29.tar.gz
#!rm deeplabv3_pascal_train_aug_2018_01_04.tar.gz
#!rm -rf setup*


--2020-11-18 05:56:56--  http://download.tensorflow.org/models/deeplabv3_mnv2_pascal_train_aug_2018_01_29.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 74.125.204.128, 2404:6800:4008:c07::80
Connecting to download.tensorflow.org (download.tensorflow.org)|74.125.204.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23882985 (23M) [application/x-tar]
Saving to: ‘deeplabv3_mnv2_pascal_train_aug_2018_01_29.tar.gz’

deeplabv3_mnv2_pasc 100%[===================>]  22.78M  11.0MB/s    in 2.1s    

2020-11-18 05:56:59 (11.0 MB/s) - ‘deeplabv3_mnv2_pascal_train_aug_2018_01_29.tar.gz’ saved [23882985/23882985]

--2020-11-18 05:56:59--  http://download.tensorflow.org/models/deeplabv3_pascal_train_aug_2018_01_04.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 108.177.125.128, 2404:6800:4008:c01::80
Connecting to download.tensorflow.org (download.tensorflow.org)|108.177.125.128|:80... connected.
HTTP request sent, awaiting res

In [ ]:
!python3 -m pip install -r absl-py==0.9.0
!python3 -m pip install -r astunparse==1.6.3
!python3 -m pip install -r cachetools==4.1.1
!python3 -m pip install -r certifi==2020.6.20
!python3 -m pip install -r chardet==3.0.4
!python3 -m pip install -r cycler==0.10.0
!python3 -m pip install -r decorator==4.4.2
!python3 -m pip install -r django==3.0.8
!python3 -m pip install -r filelock==3.0.12
!python3 -m pip install -r future==0.18.2
!python3 -m pip install -r gast==0.3.3
!python3 -m pip install -r gdown==3.11.1
!python3 -m pip install -r gluoncv==0.7.0
!python3 -m pip install -r google-auth==1.18.0
!python3 -m pip install -r google-auth-oauthlib==0.4.1
!python3 -m pip install -r google-pasta==0.2.0
!python3 -m pip install -r graphviz==0.8.4
!python3 -m pip install -r grpcio==1.30.0
!python3 -m pip install -r h5py==2.10.0
!python3 -m pip install -r idna==2.10
!python3 -m pip install -r imageio==2.9.0
!python3 -m pip install -r Keras-Preprocessing==1.1.2
!python3 -m pip install -r kiwisolver==1.2.0 
!python3 -m pip install -r Markdown==3.2.2
!python3 -m pip install -r matplotlib==3.2.2
!python3 -m pip install -r mxnet==1.6.0
!python3 -m pip install -r networkx==2.4
!python3 -m pip install -r numpy==1.19.0
!python3 -m pip install -r oauthlib==3.1.0
!python3 -m pip install -r opencv-python==4.3.0.36
!python3 -m pip install -r opt-einsum==3.2.1
!python3 -m pip install -r Pillow==7.2.0
!python3 -m pip install -r portalocker==1.7.0 
!python3 -m pip install -r protobuf==3.12.2
!python3 -m pip install -r pyasn1==0.4.8
!python3 -m pip install -r pyasn1-modules==0.2.8
!python3 -m pip install -r pyparsing==2.4.7
!python3 -m pip install -r PySocks==1.7.1
!python3 -m pip install -r python-dateutil==2.8.1
!python3 -m pip install -r PyWavelets==1.1.1
!python3 -m pip install -r requests==2.24.0
!python3 -m pip install -r requests-oauthlib==1.3.0
!python3 -m pip install -r rsa==4.6
!python3 -m pip install -r scikit-image==0.17.2
!python3 -m pip install -r scipy==1.4.1
!python3 -m pip install -r six==1.15.0
!python3 -m pip install -r tensorboard==2.2.2
!python3 -m pip install -r tensorboard-plugin-wit==1.7.0
!python3 -m pip install -r tensorflow==2.2.1
!python3 -m pip install -r tensorflow-estimator==2.2.0
!python3 -m pip install -r termcolor==1.1.0
!python3 -m pip install -r tifffile==2020.7.4
!python3 -m pip install -r torch==1.5.1
!python3 -m pip install -r tqdm==4.47.0
!python3 -m pip install -r urllib3==1.25.9
!python3 -m pip install -r Werkzeug==1.0.1
!python3 -m pip install -r wrapt==1.12.1